In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import datetime as dt
from openpyxl import load_workbook

In [13]:
pd.set_option('display.max_columns', 500)

In [2]:
#read all the valid tickers for reits from csv
wb = load_workbook(filename='yahoo_finance_reits_data.xlsx')
print(wb.sheetnames)

['PLD', 'AMT', 'EQIX', 'CCI', 'PSA', 'O', 'SPG', 'WELL', 'VICI', 'DLR', 'SBAC', 'AVB', 'EQR', 'WY', 'EXR', 'ARE', 'INVH', 'VTR', 'MAA', 'SUI', 'WPC', 'IRM', 'GLPI', 'ESS', 'UDR', 'ELS', 'PEAK', 'KIM', 'REXR', 'HST', 'CPT', 'LSI', 'AMH', 'CUBE', 'REG', 'LAMR', 'NNN', 'BXP', 'FRT', 'COLD', 'HR', 'EGP', 'FR', 'OHI', 'BRX', 'ADC', 'STAG', 'SRC', 'TRNO', 'AIRC', 'RYN', 'RHP', 'MPW', 'KRG', 'PCH', 'PECO', 'NSA', 'IRT', 'DOC', 'KRC', 'APLE', 'BNL', 'CUZ', 'LXP', 'EPR', 'VNO', 'OFC', 'PK', 'OUT', 'SITC', 'SBRA', 'FCPT', 'EQC', 'NHI', 'HIW', 'IIPR', 'MAC', 'SKT', 'DEI', 'SHO', 'CTRE', 'SAFE', 'DBRG', 'UE', 'PEB', 'GTY', 'ROIC', 'JBGS', 'RLJ', 'DRH', 'SVC', 'XHR', 'LTC', 'SLG', 'ALEX', 'AKR', 'VRE', 'GNL', 'DEA', 'AIV', 'NXRT', 'NTST', 'AAT', 'GEO', 'ALX', 'ESRT', 'FBRT', 'CHCT', 'PGRE', 'BFS', 'UMH', 'PDM', 'UNIT', 'CSR', 'CBL', 'HPP', 'RTL', 'RPT', 'AHH', 'BDN', 'BRSP', 'INN', 'UHT', 'UBA', 'UBP', 'FPI', 'LAND', 'GMRE', 'OPI', 'GOOD', 'CLDT', 'OLP', 'SRG', 'WSR', 'BRDG', 'BRT', 'ONL', 'DHC', '

In [15]:
tickers = wb.sheetnames
tickers

160

In [5]:
def backfill(df):
    # function to backfill blank or null values based on nearest available value
    for col in df:
        df[col] = (df[col].replace(0, np.nan)
                                          #.groupby(df['symbol'])
                                          .transform(lambda x: x.ffill().bfill())
                                          .fillna(0))
    return df

In [6]:
def get_quarter(value):
    current_month = float(value.split('-')[1])
    if current_month <= 3:
        return 'Q1'
    elif current_month <= 6:
        return 'Q2'
    elif current_month <= 9:
        return 'Q3'
    else: 
        return 'Q4'

In [7]:
def get_fmp_dataframe(url,attribute=None):
    response = requests.get(url)
    if attribute:
        return pd.DataFrame(response.json()[attribute])
    else:
        return pd.DataFrame(response.json())

In [8]:
def get_data(ticker):

    apikey = 'ed422f5ab8a52bef7a04a8d39de5129d'

    #Get historical stock price
    api_url5 = "https://financialmodelingprep.com/api/v3/historical-price-full/PLD?from=2019-02-01&to=2024-01-31&apikey=ed422f5ab8a52bef7a04a8d39de5129d"
    api_url5_2 = "https://financialmodelingprep.com/api/v3/historical-price-full/PLD?from=2014-02-01&to=2019-01-31&apikey=ed422f5ab8a52bef7a04a8d39de5129d"
    api_url5_3 = "https://financialmodelingprep.com/api/v3/historical-price-full/PLD?from=2009-02-01&to=2014-01-31&apikey=ed422f5ab8a52bef7a04a8d39de5129d"
    api_url5_4 = "https://financialmodelingprep.com/api/v3/historical-price-full/PLD?from=2004-02-01&to=2009-01-31&apikey=ed422f5ab8a52bef7a04a8d39de5129d"
    api_url5_5 = "https://financialmodelingprep.com/api/v3/historical-price-full/PLD?from=1999-02-01&to=2004-01-31&apikey=ed422f5ab8a52bef7a04a8d39de5129d"

    df5_1 = get_fmp_dataframe(api_url5,"historical")
    df5_2 = get_fmp_dataframe(api_url5_2,"historical")
    df5_3 = get_fmp_dataframe(api_url5_3,"historical")
    df5_4 = get_fmp_dataframe(api_url5_4,"historical")
    df5_5 = get_fmp_dataframe(api_url5_5,"historical")

    df5 = pd.concat([df5_1,df5_2,df5_3,df5_4,df5_5])
    
    #add quarters
    df5['period'] = df5['date'].apply(lambda x: get_quarter(x))
    #add year
    df5['calendarYear'] = df5['date'].apply(lambda x: x.split('-')[0])
    historicalStockPrice = backfill(df5)


    # Get Income Statement Data
    api_url = f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period=quarter&apikey={apikey}'
    response = requests.get(api_url)
    df = pd.DataFrame(response.json())
    # Join Income Statement
    incomeStatement = backfill(df.drop(columns=['date'],axis=1))
    merge1 = pd.merge(historicalStockPrice, incomeStatement[incomeStatement.columns.difference(historicalStockPrice.columns).tolist()+['period','calendarYear']],  how='left', on=['period','calendarYear'])


    # Get Balance Sheet Data
    api_url2 = f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period=quarter&apikey={apikey}'
    response2 = requests.get(api_url2)
    df2 = pd.DataFrame(response2.json())
    # Join Balance Sheet
    balanceSheet = backfill(df2.drop(columns=['date'],axis=1))
    merge2 = pd.merge(merge1, balanceSheet[balanceSheet.columns.difference(merge1.columns).tolist()+['period','calendarYear']],  how='left', on=['period','calendarYear'])


    # Get Cashflow Statement
    api_url3 = f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period=quarter&apikey={apikey}'
    response3 = requests.get(api_url3)
    df3 = pd.DataFrame(response3.json())
    # Join Cash Flow Statement
    cashFlowStatement = backfill(df3.drop(columns=['date'],axis=1))
    merge3 = pd.merge(merge2, cashFlowStatement[cashFlowStatement.columns.difference(merge2.columns).tolist()+['period','calendarYear']],  how='left', on=['period','calendarYear'])


    # Get Financial Statement Analysis
    api_url4 = f'https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period=quarter&apikey={apikey}'
    response4 = requests.get(api_url4)
    df4 = pd.DataFrame(response4.json())
    # Join Financial Analysis
    FSA = backfill(df4.drop(columns=['date'],axis=1))
    merge4 = pd.merge(merge3, FSA[FSA.columns.difference(merge3.columns).tolist()+['period','calendarYear']],  how='left', on=['period','calendarYear'])
    
    merge4['date'] = pd.to_datetime(merge4['date'])
    
    return merge4



In [9]:
pld = get_data(tickers[0])

In [14]:
pld.dropna()

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,period,calendarYear,acceptedDate,cik,costAndExpenses,costOfRevenue,depreciationAndAmortization,ebitda,ebitdaratio,eps,epsdiluted,fillingDate,finalLink,generalAndAdministrativeExpenses,grossProfit,grossProfitRatio,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,interestExpense,interestIncome,link,netIncome,netIncomeRatio,operatingExpenses,operatingIncome,operatingIncomeRatio,otherExpenses,reportedCurrency,researchAndDevelopmentExpenses,revenue,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,symbol,totalOtherIncomeExpensesNet,weightedAverageShsOut,weightedAverageShsOutDil,accountPayables,accumulatedOtherComprehensiveIncomeLoss,capitalLeaseObligations,cashAndCashEquivalents,cashAndShortTermInvestments,commonStock,deferredRevenue,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,goodwill,goodwillAndIntangibleAssets,intangibleAssets,inventory,longTermDebt,longTermInvestments,minorityInterest,netDebt,netReceivables,otherAssets,otherCurrentAssets,otherCurrentLiabilities,otherLiabilities,otherNonCurrentAssets,otherNonCurrentLiabilities,othertotalStockholdersEquity,preferredStock,propertyPlantEquipmentNet,retainedEarnings,shortTermDebt,shortTermInvestments,taxAssets,taxPayables,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalDebt,totalEquity,totalInvestments,totalLiabilities,totalLiabilitiesAndStockholdersEquity,totalLiabilitiesAndTotalEquity,totalNonCurrentAssets,totalNonCurrentLiabilities,totalStockholdersEquity,accountsPayables,accountsReceivables,acquisitionsNet,capitalExpenditure,cashAtBeginningOfPeriod,cashAtEndOfPeriod,changeInWorkingCapital,commonStockIssued,commonStockRepurchased,debtRepayment,deferredIncomeTax,dividendsPaid,effectOfForexChangesOnCash,freeCashFlow,investmentsInPropertyPlantAndEquipment,netCashProvidedByOperatingActivities,netCashUsedForInvestingActivites,netCashUsedProvidedByFinancingActivities,netChangeInCash,operatingCashFlow,otherFinancingActivites,otherInvestingActivites,otherNonCashItems,otherWorkingCapital,purchasesOfInvestments,salesMaturitiesOfInvestments,stockBasedCompensation,averageInventory,averagePayables,averageReceivables,bookValuePerShare,capexPerShare,capexToDepreciation,capexToOperatingCashFlow,capexToRevenue,cashPerShare,currentRatio,daysOfInventoryOnHand,daysPayablesOutstanding,daysSalesOutstanding,debtToAssets,debtToEquity,dividendYield,earningsYield,enterpriseValue,enterpriseValueOverEBITDA,evToFreeCashFlow,evToOperatingCashFlow,evToSales,freeCashFlowPerShare,freeCashFlowYield,grahamNetNet,grahamNumber,incomeQuality,intangiblesToTotalAssets,interestCoverage,interestDebtPerShare,inventoryTurnover,investedCapital,marketCap,netCurrentAssetValue,netDebtToEBITDA,netIncomePerShare,operatingCashFlowPerShare,payablesTurnover,payoutRatio,pbRatio,peRatio,pfcfRatio,pocfratio,priceToSalesRatio,ptbRatio,receivablesTurnover,researchAndDdevelopementToRevenue,returnOnTangibleAssets,revenuePerShare,roe,roic,salesGeneralAndAdministrativeToRevenue,shareholdersEquityPerShare,stockBasedCompensationToRevenue,tangibleAssetValue,tangibleBookValuePerShare,workingCapital
21,2023-12-29,134.08,135.04,133.07,133.30,133.30,2938090,2938090,-0.78,-0.58174,133.8725,"December 29, 23",-0.005817,Q4,2023,2024-02-13 16:06:27,0001045609,1.281867e+09,1.125429e+09,638346000.0,1.245726e+09,0.659377,0.66,0.66,2024-02-13,https://www.sec.gov/Archives/edgar/data/104560...,96673000.0,763818000.0,0.404298,734381000.0,0.388716,58497000.0,212070000.0,181053000.0,https://www.sec.gov/Archives/edgar/data/104560...,630936000.0,0.333962,156438000.0,607380000.0,0.321493,-156438000.0,USD,17820000.0,1.889247e+09,146000.0,96673000.0,PLD,127001000.0,952399000.0,952399000.0,1.721108e+09,-523445000.0,2.804469e+09,530388000.0,530388000.0,9244000.0,-1.721108e+09,347369000.0,-3.146533e+09,25280000.0,836600000.0,836600000.0,-877333000.0,2.846998e+10,9.631289e+09,4.641996e+09,3.127458e+10,554337000.0,9.3

In [22]:
tickers = tickers[14:]

In [23]:
for stock in tickers:
    stockdata = get_data(stock)
    output = stockdata.dropna()
    output.to_csv('data/'+ stock + '.csv')